In [3]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import tensorflow as tf
import matplotlib
import os
import matplotlib.pyplot as plt
from datetime import datetime
tf.set_random_seed(77)

mockForecastDictionary = {}
realForecastDictionary = {}

def LearningModuleRunner(rawArrayDatas, processId, forecastDay,dayOrWeekOrMonth):
    #TODO make dayOrWeekOrMonth parameter
    dayOrWeekOrMonth=dayOrWeekOrMonth
    # options:
    # 'day', 'week', 'month'

    feature = 'DayOfWeek_WeekNumber_Month_Season'
    # options:
    # dayOrWeekOrMonth='day': 'DayOfWeek_WeekNumber_Month_Season','DayOfWeek01_WeekNumber_Month_Season'//
    # dayOrWeekOrMonth='week': 'WeekNumber_Month_Season_Year'

#     LoggingManager.PrintLogMessage("LearningManager", "LearningModuleRunner", "start of learning #" + str(processId), DefineManager.LOG_LEVEL_INFO)

    global mockForecastDictionary
    global realForecastDictionary
    mockForcastDay=forecastDay

    ##Make txsForRealForecastLstm   [:]
    ds = rawArrayDatas[0]
    y = list(np.sqrt(rawArrayDatas[1]))
    sales = list(zip(ds, y))
    txsForRealForecastLstm =pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForMockForecastLstm [:-forecastDay]
    ds = rawArrayDatas[0][:-forecastDay]
    y= list(np.sqrt(rawArrayDatas[1][:-forecastDay] ))
    sales = list(zip(ds, y))
    txsForMockForecastLstm =pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForRealForecastBayesian [:-forecastDay] & np.log
    ds = rawArrayDatas[0][:-forecastDay]
    # TODO bayseian에 대해서는 input값이 0인 상황처리 필요
    y = list(np.log(rawArrayDatas[1][:-forecastDay]))
    sales = list(zip(ds, y))
    txsForRealForecastBayesian = pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForMockForecastBayseian   [:-(mockForcastDay+forecastDay)] & np.log
    ds = rawArrayDatas[0][:-(mockForcastDay+forecastDay)]
    #TODO bayseian에 대해서는 input값이 0인 상황처리 필요
    y= list(np.log(rawArrayDatas[1][:-(mockForcastDay+forecastDay)]))
    sales = list(zip(ds, y))
    txsForMockForecastBayseian =pd.DataFrame(data=sales, columns=['ds', 'y'])

    #testY for algorithm compare has size of (mockForcastDay+forecastDay)  rawArrayDatas[1][-(mockForcastDay+forecastDay):-forecastDay]
    testY= rawArrayDatas[1][-(mockForcastDay+forecastDay):-forecastDay]


    if dayOrWeekOrMonth is 'day':
        ####LSTM_day

        #select feature module
        feature='DayOfWeek_WeekNumber_Month_Season'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay,feature)

      
        ####Bayseian_day

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'day')

        #알고리즘 비교
        nameOfBestAlgorithm= AlgorithmCompare(testY)
        
        ####더 좋은 알고리즘 호출       
        tf.reset_default_graph()
        realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)
        realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'day')
            
    elif dayOrWeekOrMonth is 'week':
        
        ####LSTM_week

        # select feature module
        feature = 'WeekNumber_Month_Season_Year'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay, feature)


        ####Bayseian_week

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'week')


        # 알고리즘 비교
        nameOfBestAlgorithm = AlgorithmCompare(testY)
        
        ####더 좋은 알고리즘 호출
        tf.reset_default_graph()
        realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)
        realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'week')
    
            
    elif dayOrWeekOrMonth is 'month':
       
        ####LSTM_month

        # select feature module
        feature = 'WeekNumber_Month_Season_Year'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay, feature)    

        ####Bayseian_month

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'month')


        # 알고리즘 비교
        nameOfBestAlgorithm = AlgorithmCompare(testY)
        
        ####더 좋은 알고리즘 호출
        tf.reset_default_graph()
        realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)
        realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'month')

        ####################################################################################BAYSEIAN
    print('LSTM realforecast :',realForecastDictionary['LSTM'])
    print('Bayseian realforecast :',realForecastDictionary['Bayseian'] ) 
    data = rawArrayDatas[1][:-forecastDay] + realForecastDictionary[nameOfBestAlgorithm]
    date= rawArrayDatas[0]
    print("===================================THE END===================================================")
    return realForecastDictionary[nameOfBestAlgorithm]


def LSTM(txs, forecastDay, features):
    tf.reset_default_graph()
    tf.set_random_seed(77)
    # Add basic date related features to the table
    year = lambda x: datetime.strptime(x, "%Y-%m-%d").year
    dayOfWeek = lambda x: datetime.strptime(x, "%Y-%m-%d").weekday()
    month = lambda x: datetime.strptime(x, "%Y-%m-%d").month
    weekNumber = lambda x: datetime.strptime(x, "%Y-%m-%d").strftime('%V')
    txs['year'] = txs['ds'].map(year)
    txs['month'] = txs['ds'].map(month)
    txs['weekNumber'] = txs['ds'].map(weekNumber)
    txs['dayOfWeek'] = txs['ds'].map(dayOfWeek)

    # Add non-basic date related features to the table
    seasons = [0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 0]  # dec - feb is winter, then spring, summer, fall etc
    season = lambda x: seasons[(datetime.strptime(x, "%Y-%m-%d").month - 1)]
    day_of_week01s = [0, 0, 0, 0, 0, 1, 1]
    day_of_week01 = lambda x: day_of_week01s[(datetime.strptime(x, "%Y-%m-%d").weekday())]
    txs['season'] = txs['ds'].map(season)
    txs['dayOfWeek01'] = txs['ds'].map(day_of_week01)

    # Backup originalSales
    originalSales = list(txs['y'])
    sales = list(txs['y'])
    
    #week number는 경계부분에서 약간 오류가 있다.
    if features is 'DayOfWeek_WeekNumber_Month_Season':
        tempxy = [list(txs['dayOfWeek']), list(txs['weekNumber']), list(txs['month']), list(txs['season']), sales]
    elif features is 'DayOfWeek01_WeekNumber_Month_Season':
        tempxy = [list(txs['dayOfWeek01']), list(txs['weekNumber']), list(txs['month']), list(txs['season']), sales]

    elif features is 'WeekNumber_Month_Season_Year':
        tempxy = [list(txs['weekNumber']), list(txs['month']), list(txs['season']), list(txs['year']), sales]
    
   
    xy = np.array(tempxy).transpose().astype(np.float)
    


    # Backup originalXY for denormalize
    originalXY = np.array(tempxy).transpose().astype(np.float)
    xy = minMaxNormalizer(xy)

    # TRAIN PARAMETERS
    # data_dim은 y값 도출을 위한 feature 가지수+1(독립변수 가지수 +1(y포함))
    data_dim = 5
    # data_dim크기의 data 한 묶음이 seq_length만큼 input으로 들어가
    seq_length = 10
    # output_dim(=forecastDays)만큼의 다음날 y_data를 예측
    output_dim = forecastDay
    # hidden_dim은 정말 임의로 설정
    hidden_dim = 100
    # learning rate은 배우는 속도(너무 크지도, 작지도 않게 설정)
    learning_rate = 0.001
    iterations=1000
    # Build a series dataset(seq_length에 해당하는 전날 X와 다음 forecastDays에 해당하는 Y)
    x = xy
    y = xy[:, [-1]]
    dataX = []
    dataY = []
    for i in range(0, len(y) - seq_length - forecastDay + 1):
        _x = x[i:i + seq_length]
        _y = y[i + seq_length:i + seq_length + forecastDay]
        _y = np.reshape(_y, (forecastDay))
        dataX.append(_x)
        dataY.append(_y)

    train_size = int(len(dataY) - forecastDay)
    # train_size = int(len(dataY) * 0.7)
    test_size = len(dataY) - train_size

    trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:])

    trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:])

    X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
    Y = tf.placeholder(tf.float32, [None, forecastDay])

    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
    outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs[:, -1], output_dim, activation_fn=None)
    loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train = optimizer.minimize(loss)

    denormalizedTestY = originalSales[train_size + seq_length:]
    #     denormalizedTestY_feed=np.array([[i] for i in denormalizedTestY])

    targets = tf.placeholder(tf.float32, [None, 1])
    predictions = tf.placeholder(tf.float32, [None, 1])

    count = 0
    with tf.Session() as sess:

        # 초기화
        init = tf.global_variables_initializer()
        sess.run(init)

        # Training step
        for i in range(iterations):

            _, step_loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
            print("[step: {}] loss: {}".format(i, step_loss))


        # Test step
        test_predict = minMaxDeNormalizer(sess.run(Y_pred, feed_dict={X: testX}), originalXY)
        realSale = minMaxDeNormalizer(testY[-1], originalXY)
#         listedLogPredict=test_predict[-1].tolist()
#     return [np.exp(y) for y in listedLogPredict]
    return np.square(test_predict[-1]).tolist()

def Bayseian(txs, forecastDay, unit):
    global mockForecastDictionary
    global realForecastDictionary

    if unit is 'day':
        if (len(txs) < 366):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay)
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay)
            forecastProphetTable = model.predict(future)


    elif unit is 'week':
        if(len(txs)<53):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='w')
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='w')
            forecastProphetTable = model.predict(future)

    elif unit is 'month':
        if(len(txs)<12):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='m')
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='m')
            forecastProphetTable = model.predict(future)


    #date = [d.strftime('%Y-%m-%d') for d in forecastProphetTable['ds']]
    return [np.exp(y) for y in forecastProphetTable['yhat'][-forecastDay:]]

def rmse(a,b):
    sum=0
    for i in range(len(a)):
        sum=sum+(a[i]-b[i])**2
    return np.sqrt(sum/len(a))

def minMaxNormalizer(data):
    numerator=data-np.min(data)
    denominator=np.max(data)-np.min(data)
    return numerator/(denominator+1e-7)

def minMaxDeNormalizer(data, originalData):
    shift=np.min(originalData)
    multiplier=np.max(originalData)-np.min(originalData)
    return (data+shift)*multiplier

def AlgorithmCompare(testY):
    global mockForecastDictionary
    nameOfBestAlgorithm = 'LSTM'
    minData = rmse(testY, mockForecastDictionary[nameOfBestAlgorithm])
    rms = 0
    for algorithm in mockForecastDictionary.keys():
        rms = rmse(testY, mockForecastDictionary[algorithm])
        if rms < minData:
            nameOfBestAlgorithm = algorithm
    print('testY is: ', testY)
    print('\n')
    print('LSTM testforecast :',mockForecastDictionary['LSTM'] , '\n@@@@@LSTM rmse: ',rmse(testY, mockForecastDictionary['LSTM']) )
    print('Bayseian testforecast :',mockForecastDictionary['Bayseian'], '\n@@@@@Bayseian rmse: ',rmse(testY, mockForecastDictionary['Bayseian']) ) 
    print('\n')
    print(nameOfBestAlgorithm, 'WON!!!!!!')
    return nameOfBestAlgorithm


In [4]:
# def LSTM(txs, forecastDay, features)
# def Bayseian(txs, forecastDay, unit)

In [5]:
columns=['ds','y']
df=pd.read_table('webMonth.csv', sep=',',header=None,names=columns )
forecastDay=int(len(df)*0.2)
print(len(df), forecastDay)


98 19


In [6]:
df['y'][-forecastDay:]=[1]*forecastDay
rawArrayDatas=[[i for i in df['ds']],[i for i in df['y']]]

C:\Users\User\Anaconda3\envs\tensorflow35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
rawArrayDatas

[['2007-12-01',
  '2008-01-01',
  '2008-02-01',
  '2008-03-01',
  '2008-04-01',
  '2008-05-01',
  '2008-06-01',
  '2008-07-01',
  '2008-08-01',
  '2008-09-01',
  '2008-10-01',
  '2008-11-01',
  '2008-12-01',
  '2009-01-01',
  '2009-02-01',
  '2009-03-01',
  '2009-04-01',
  '2009-05-01',
  '2009-06-01',
  '2009-07-01',
  '2009-08-01',
  '2009-09-01',
  '2009-10-01',
  '2009-11-01',
  '2009-12-01',
  '2010-01-01',
  '2010-02-01',
  '2010-03-01',
  '2010-04-01',
  '2010-05-01',
  '2010-06-01',
  '2010-07-01',
  '2010-08-01',
  '2010-09-01',
  '2010-10-01',
  '2010-11-01',
  '2010-12-01',
  '2011-01-01',
  '2011-02-01',
  '2011-03-01',
  '2011-04-01',
  '2011-05-01',
  '2011-06-01',
  '2011-07-01',
  '2011-08-01',
  '2011-09-01',
  '2011-10-01',
  '2011-11-01',
  '2011-12-01',
  '2012-01-01',
  '2012-02-01',
  '2012-03-01',
  '2012-04-01',
  '2012-05-01',
  '2012-06-01',
  '2012-07-01',
  '2012-08-01',
  '2012-09-01',
  '2012-10-01',
  '2012-11-01',
  '2012-12-01',
  '2013-01-01',
  '2013-

In [8]:
answer=LearningModuleRunner(rawArrayDatas,0,forecastDay,'month') #0은 processId

[step: 0] loss: 4.056002616882324
[step: 1] loss: 2.239607810974121
[step: 2] loss: 1.0944609642028809
[step: 3] loss: 0.4757412075996399
[step: 4] loss: 0.25414401292800903
[step: 5] loss: 0.2867094874382019
[step: 6] loss: 0.42198461294174194
[step: 7] loss: 0.5423859357833862
[step: 8] loss: 0.5898133516311646
[step: 9] loss: 0.5613851547241211
[step: 10] loss: 0.4848596453666687
[step: 11] loss: 0.39340585470199585
[step: 12] loss: 0.31138330698013306
[step: 13] loss: 0.2506399154663086
[step: 14] loss: 0.21273574233055115
[step: 15] loss: 0.19317218661308289
[step: 16] loss: 0.18540246784687042
[step: 17] loss: 0.1834794580936432
[step: 18] loss: 0.18329213559627533
[step: 19] loss: 0.18278414011001587
[step: 20] loss: 0.18151724338531494
[step: 21] loss: 0.17991620302200317
[step: 22] loss: 0.1785127818584442
[step: 23] loss: 0.17743057012557983
[step: 24] loss: 0.17625868320465088
[step: 25] loss: 0.17428365349769592
[step: 26] loss: 0.1708613932132721
[step: 27] loss: 0.1657021

[step: 255] loss: 0.10957527905702591
[step: 256] loss: 0.10943381488323212
[step: 257] loss: 0.10928872972726822
[step: 258] loss: 0.10913988947868347
[step: 259] loss: 0.10898715257644653
[step: 260] loss: 0.10883030295372009
[step: 261] loss: 0.1086692065000534
[step: 262] loss: 0.10850364714860916
[step: 263] loss: 0.10833350569009781
[step: 264] loss: 0.10815852880477905
[step: 265] loss: 0.10797859728336334
[step: 266] loss: 0.107793428003788
[step: 267] loss: 0.10760285705327988
[step: 268] loss: 0.10740667581558228
[step: 269] loss: 0.1072046309709549
[step: 270] loss: 0.10699652880430222
[step: 271] loss: 0.10678210109472275
[step: 272] loss: 0.10656111687421799
[step: 273] loss: 0.10633335262537003
[step: 274] loss: 0.10609851032495499
[step: 275] loss: 0.10585635900497437
[step: 276] loss: 0.10560666024684906
[step: 277] loss: 0.10534914582967758
[step: 278] loss: 0.10508349537849426
[step: 279] loss: 0.10480953752994537
[step: 280] loss: 0.10452690720558167
[step: 281] loss

[step: 505] loss: 0.05779936909675598
[step: 506] loss: 0.057600103318691254
[step: 507] loss: 0.057839058339595795
[step: 508] loss: 0.05810234695672989
[step: 509] loss: 0.05805432051420212
[step: 510] loss: 0.05775976553559303
[step: 511] loss: 0.05749909207224846
[step: 512] loss: 0.05745594948530197
[step: 513] loss: 0.05758638307452202
[step: 514] loss: 0.05771888419985771
[step: 515] loss: 0.05772031843662262
[step: 516] loss: 0.057580962777137756
[step: 517] loss: 0.05739665776491165
[step: 518] loss: 0.05727636441588402
[step: 519] loss: 0.05726403743028641
[step: 520] loss: 0.057319968938827515
[step: 521] loss: 0.05736570432782173
[step: 522] loss: 0.057346656918525696
[step: 523] loss: 0.05726398527622223
[step: 524] loss: 0.05716385319828987
[step: 525] loss: 0.05709563195705414
[step: 526] loss: 0.05707792937755585
[step: 527] loss: 0.05709299445152283
[step: 528] loss: 0.05710697919130325
[step: 529] loss: 0.057095255702733994
[step: 530] loss: 0.0570540726184845
[step: 

[step: 761] loss: 0.05648218095302582
[step: 762] loss: 0.05531848594546318
[step: 763] loss: 0.05380221828818321
[step: 764] loss: 0.05435928702354431
[step: 765] loss: 0.05536745488643646
[step: 766] loss: 0.05472143366932869
[step: 767] loss: 0.0537002831697464
[step: 768] loss: 0.054037488996982574
[step: 769] loss: 0.05472887307405472
[step: 770] loss: 0.054368432611227036
[step: 771] loss: 0.053621985018253326
[step: 772] loss: 0.053646381944417953
[step: 773] loss: 0.05416981875896454
[step: 774] loss: 0.05426378548145294
[step: 775] loss: 0.05380944162607193
[step: 776] loss: 0.05344036966562271
[step: 777] loss: 0.05353865772485733
[step: 778] loss: 0.0538615956902504
[step: 779] loss: 0.05398765206336975
[step: 780] loss: 0.05379386991262436
[step: 781] loss: 0.053494229912757874
[step: 782] loss: 0.05334855616092682
[step: 783] loss: 0.05342436581850052
[step: 784] loss: 0.05358676612377167
[step: 785] loss: 0.05365859717130661
[step: 786] loss: 0.05357322841882706
[step: 78

[step: 978] loss: 0.050939977169036865
[step: 979] loss: 0.05090762674808502
[step: 980] loss: 0.050883810967206955
[step: 981] loss: 0.050868891179561615
[step: 982] loss: 0.05086095258593559
[step: 983] loss: 0.05085720866918564
[step: 984] loss: 0.05085498094558716
[step: 985] loss: 0.050852470099925995
[step: 986] loss: 0.05084867775440216
[step: 987] loss: 0.05084335058927536
[step: 988] loss: 0.050836820155382156
[step: 989] loss: 0.05082953721284866
[step: 990] loss: 0.050822190940380096
[step: 991] loss: 0.05081547796726227
[step: 992] loss: 0.05081011727452278
[step: 993] loss: 0.05080687254667282
[step: 994] loss: 0.050806738436222076
[step: 995] loss: 0.05081078037619591
[step: 996] loss: 0.05082052946090698
[step: 997] loss: 0.05083779990673065
[step: 998] loss: 0.0508652925491333
[step: 999] loss: 0.050906188786029816
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
testY is:  [9169.16129, 13726.903230000002, 4432.5, 2776.51612900000

[step: 175] loss: 0.22804880142211914
[step: 176] loss: 0.22796298563480377
[step: 177] loss: 0.2278764545917511
[step: 178] loss: 0.2277892827987671
[step: 179] loss: 0.22770142555236816
[step: 180] loss: 0.22761297225952148
[step: 181] loss: 0.22752374410629272
[step: 182] loss: 0.22743387520313263
[step: 183] loss: 0.22734323143959045
[step: 184] loss: 0.22725197672843933
[step: 185] loss: 0.22715990245342255
[step: 186] loss: 0.2270670235157013
[step: 187] loss: 0.22697344422340393
[step: 188] loss: 0.22687901556491852
[step: 189] loss: 0.2267838567495346
[step: 190] loss: 0.22668775916099548
[step: 191] loss: 0.2265908271074295
[step: 192] loss: 0.22649309039115906
[step: 193] loss: 0.22639445960521698
[step: 194] loss: 0.22629477083683014
[step: 195] loss: 0.2261943221092224
[step: 196] loss: 0.2260928452014923
[step: 197] loss: 0.22599036991596222
[step: 198] loss: 0.22588686645030975
[step: 199] loss: 0.2257823497056961
[step: 200] loss: 0.2256767898797989
[step: 201] loss: 0.2

[step: 405] loss: 0.13422566652297974
[step: 406] loss: 0.133572518825531
[step: 407] loss: 0.13305196166038513
[step: 408] loss: 0.13277874886989594
[step: 409] loss: 0.13275229930877686
[step: 410] loss: 0.13287527859210968
[step: 411] loss: 0.13301485776901245
[step: 412] loss: 0.13305790722370148
[step: 413] loss: 0.13296383619308472
[step: 414] loss: 0.1327427625656128
[step: 415] loss: 0.13246284425258636
[step: 416] loss: 0.13219018280506134
[step: 417] loss: 0.13197726011276245
[step: 418] loss: 0.1318414956331253
[step: 419] loss: 0.1317743957042694
[step: 420] loss: 0.1317553073167801
[step: 421] loss: 0.13176418840885162
[step: 422] loss: 0.13179093599319458
[step: 423] loss: 0.13183307647705078
[step: 424] loss: 0.13190260529518127
[step: 425] loss: 0.13201238214969635
[step: 426] loss: 0.13219690322875977
[step: 427] loss: 0.1324787586927414
[step: 428] loss: 0.13291724026203156
[step: 429] loss: 0.13350847363471985
[step: 430] loss: 0.13427603244781494
[step: 431] loss: 0

[step: 645] loss: 0.12338551878929138
[step: 646] loss: 0.12338059395551682
[step: 647] loss: 0.12338020652532578
[step: 648] loss: 0.12338327616453171
[step: 649] loss: 0.12338974326848984
[step: 650] loss: 0.12340271472930908
[step: 651] loss: 0.12342531979084015
[step: 652] loss: 0.12346664071083069
[step: 653] loss: 0.12353537976741791
[step: 654] loss: 0.12365471571683884
[step: 655] loss: 0.12384700030088425
[step: 656] loss: 0.12417268007993698
[step: 657] loss: 0.12468039989471436
[step: 658] loss: 0.12551188468933105
[step: 659] loss: 0.12668950855731964
[step: 660] loss: 0.12838876247406006
[step: 661] loss: 0.13008779287338257
[step: 662] loss: 0.13143512606620789
[step: 663] loss: 0.13064612448215485
[step: 664] loss: 0.12787401676177979
[step: 665] loss: 0.1243295818567276
[step: 666] loss: 0.12305742502212524
[step: 667] loss: 0.12456370145082474
[step: 668] loss: 0.12609688937664032
[step: 669] loss: 0.12539632618427277
[step: 670] loss: 0.123502716422081
[step: 671] los

[step: 883] loss: 0.12343788892030716
[step: 884] loss: 0.12513284385204315
[step: 885] loss: 0.1269608587026596
[step: 886] loss: 0.12717869877815247
[step: 887] loss: 0.12562884390354156
[step: 888] loss: 0.12196696549654007
[step: 889] loss: 0.11917870491743088
[step: 890] loss: 0.1193016916513443
[step: 891] loss: 0.12124939262866974
[step: 892] loss: 0.12199696153402328
[step: 893] loss: 0.12040000408887863
[step: 894] loss: 0.11890602856874466
[step: 895] loss: 0.11932443082332611
[step: 896] loss: 0.12048456817865372
[step: 897] loss: 0.12048039585351944
[step: 898] loss: 0.11932242661714554
[step: 899] loss: 0.11864006519317627
[step: 900] loss: 0.11906571686267853
[step: 901] loss: 0.11976911872625351
[step: 902] loss: 0.11985558271408081
[step: 903] loss: 0.1192525252699852
[step: 904] loss: 0.11863572895526886
[step: 905] loss: 0.1184849739074707
[step: 906] loss: 0.11878585815429688
[step: 907] loss: 0.11919897794723511
[step: 908] loss: 0.11938706040382385
[step: 909] loss

In [9]:
len(answer)

19

In [10]:
answer

[4260.25,
 7416.30810546875,
 10204.5029296875,
 12543.4189453125,
 12502.0380859375,
 10352.0888671875,
 6890.15869140625,
 4086.7216796875,
 2252.292236328125,
 1619.9642333984375,
 1773.136474609375,
 2929.975341796875,
 4677.8828125,
 7465.01171875,
 12251.361328125,
 15952.8505859375,
 16215.6162109375,
 13491.8291015625,
 9029.412109375]

In [11]:
len(rawArrayDatas[1])

98